### Load imports and run the no LID case:

In [4]:
%reset
import yaml
from swmm_objects import *
from swmm_read import *
from swmm_run import runswmm
from subprocess import call
from pymongo import MongoClient
from datetime import datetime
import sys

#  IMPORTANT:  Set these for each run:
#################################################
mongoServer = None   # use the local server
# mongoServer = 'mongodb://server.mcgarity.info:27017/'
database = "swmm_wingo"
collection = 'y16m01d28_loop_run_1'
client = MongoClient(mongoServer)
swmmInpFile = "Example2_LID_BR_for_RG_PHL.inp"  # initial SWMM .inp file
lidZeroFile = "lidlist_zero.yaml"
################################################

db = client[database]   # use or create the database
thisRunCollection = db[collection]    # use or create the collection
infile = open(swmmInpFile,'r')
swmmInpStr = infile.readlines()
infile.close()
f = open(lidZeroFile,'r')   # Read the no LID changable parameters from file
zeroLidList = yaml.load(f)  # list of dicts, one for each line in [LID_USAGE]
f.close()
zeroLidDocId = runswmm(zeroLidList,swmmInpStr,thisRunCollection)  # run SWMM on the no LID model for baseline
cursor = thisRunCollection.find_one({'_id':zeroLidDocId})
zeroLidPeak = cursor['peak']
zeroLidVolume = cursor['volume']
print "Stored record %s for zero LID usage:  peak = %s, volume = %s" % (zeroLidDocId,zeroLidPeak,zeroLidVolume)

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
0.0 min, 12.55 sec
Stored record 56a54290c10157aeb0be2976 for zero LID usage:  peak = 32.82, volume = 15.972


### Loop ALL subcatchment LID numbers over a range:

In [6]:
from copy import deepcopy
for numberForAllLid in range(1,11):
    newRunList=deepcopy(zeroLidList)  # copy the zeroLidList into a new list
    for lid in newRunList:
        lid['Number'] = numberForAllLid  # the same number of LIDs will be in every subcatchment
    newRunDocId = runswmm(newRunList,swmmInpStr,thisRunCollection)
    cursor = thisRunCollection.find_one({'_id':newRunDocId})
    newPeak = cursor['peak']
    newVolume = cursor['volume']
    peakReduction = zeroLidPeak - newPeak
    volumeReduction = zeroLidVolume - newVolume
    print "Stored record %s for new parameters: peak = %s, volume = %s" % (newRunDocId,newPeak,newVolume)
    print "peakReduction = %s, volumeReduction = %s" % (peakReduction,volumeReduction)
print "FINISHED ALL RUNS"

0.0 min, 14.32 sec
Stored record 56a542f3c10157aeb0be2978 for new parameters: peak = 32.79, volume = 14.479
peakReduction = 0.03, volumeReduction = 1.493
0.0 min, 14.15 sec
Stored record 56a54301c10157aeb0be2979 for new parameters: peak = 32.09, volume = 14.081
peakReduction = 0.73, volumeReduction = 1.891
0.0 min, 13.99 sec
Stored record 56a5430fc10157aeb0be297a for new parameters: peak = 29.86, volume = 13.914
peakReduction = 2.96, volumeReduction = 2.058
0.0 min, 14.61 sec
Stored record 56a5431dc10157aeb0be297b for new parameters: peak = 29.76, volume = 13.829
peakReduction = 3.06, volumeReduction = 2.143
0.0 min, 14.28 sec
Stored record 56a5432cc10157aeb0be297c for new parameters: peak = 29.66, volume = 13.774
peakReduction = 3.16, volumeReduction = 2.198
0.0 min, 13.97 sec
Stored record 56a5433ac10157aeb0be297d for new parameters: peak = 29.57, volume = 13.719
peakReduction = 3.25, volumeReduction = 2.253
0.0 min, 14.47 sec
Stored record 56a54348c10157aeb0be297e for new parameters

### To display all of the runs from the database:

In [9]:
runs = db[collection]
cursor = runs.find()
for run in cursor:
    print run
    print ''

{u'lidDict': None, u'swmmStartTime': u'2016-01-24 16:30:43.453499', u'swmmInputFileStr': u'[TITLE]\nExample 2\r\nSurface Drainage System\r\n[OPTIONS]\nFLOW_UNITS\tCFS \nINFILTRATION\tHORTON \nFLOW_ROUTING\tKINWAVE \nLINK_OFFSETS\tDEPTH \nMIN_SLOPE\t0 \nALLOW_PONDING\tNO \nSKIP_STEADY_STATE\tNO \nSTART_DATE\t01/01/2008 \nSTART_TIME\t00:00:00 \nREPORT_START_DATE\t01/01/2008 \nREPORT_START_TIME\t00:00:00 \nEND_DATE\t01/01/2009 \nEND_TIME\t00:00:00 \nSWEEP_START\t01/01 \nSWEEP_END\t12/31 \nDRY_DAYS\t0 \nREPORT_STEP\t00:01:00 \nWET_STEP\t00:01:00 \nDRY_STEP\t01:00:00 \nROUTING_STEP\t0:00:15 \nINERTIAL_DAMPING\tPARTIAL \nNORMAL_FLOW_LIMITED\tSLOPE \nFORCE_MAIN_EQUATION\tH-W \nVARIABLE_STEP\t0.75 \nLENGTHENING_STEP\t0 \nMIN_SURFAREA\t0 \nMAX_TRIALS\t0 \nHEAD_TOLERANCE\t0 \nSYS_FLOW_TOL\t5 \nLAT_FLOW_TOL\t5 \nMINIMUM_STEP\t0.5 \nTHREADS\t1 \n[EVAPORATION]\nTEMPERATURE\t\nDRY_ONLY\tNO \n[TEMPERATURE]\nFILE\t"668594.dat" \nWINDSPEED\tFILE \nSNOWMELT\t34 0.5 0.6 0.0 50.0 0.0 \nADC\tPERVIOUS 1.0 1

### To Reconstitute the SWMM input file for any of the runs above

In [12]:
runs = db[collection]
cursor = runs.find_one({'_id':zeroLidDocId})
#print cursor
swmmInputFileStr = cursor["swmmInputFileStr"]
f = open("SWMM_reconstituted.inp",'w')
f.write(swmmInputFileStr)  # write out the swmmInputFileStr for modified problem
f.close()